## Input priocessing

In [27]:
import cv2
import numpy as np
from cv2 import imread
from cv2 import imshow
from cv2 import waitKey
from cv2 import destroyAllWindows
from cv2 import CascadeClassifier
from cv2 import rectangle
import torch
from FECNet import FECNet
# function to get face image from img_path

def preprocess_image_face(image_path):
  # load the photograph
  pixels = imread(image_path)
  # load the pre-trained model
  classifier = CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')
  # perform face detection
  bboxes = classifier.detectMultiScale(pixels)
  # print bounding box for each detected face
  for box in bboxes:
	  # extract
	  x, y, width, height = box
	  x2, y2 = x + width, y + height
	  # draw a rectangle over the pixels
	  #rectangle(pixels, (x, y), (x2, y2), (0,0,255), 1)
  # show the image
  #cv2_imshow(pixels)

  def get_crop_of_face(bboxes, pixels):

    list_width_height = list(np.sum(bboxes[:, -2:], axis = -1))
    index_max_width_height = list_width_height.index(max(list_width_height))

    x, y, w, h = bboxes[index_max_width_height]
    x2, y2 = x + w, y + h
    height, width, _ = pixels.shape
    cropped_img = pixels[(y-int(0.1*height)):(y2+int(0.1*height)), (x-int(0.01*width)):(x2+int(0.01*width))]
    

    cropped_img = cv2.resize(cropped_img, (224, 224))
    #cv2_imshow(cropped_img)
    x = cropped_img.astype(np.float32).reshape(-1, 3, 224, 224)
    #x = np.expand_dims(cropped_img, axis=0)
    #x = preprocess_input(x)
  
    return x

  return get_crop_of_face(bboxes, pixels)

In [28]:
def get_embeddings(img_path):
  model = FECNet()
  model.load_state_dict(torch.load('/content/drive/MyDrive/facenet model/FECNet.pt',  map_location=torch.device('cpu')))
  model.eval()
  image = preprocess_image_face(img_path)
  #print(np.shape(image))
  image = torch.from_numpy(image)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  image = image.to(device)
  #image.unsqueeze_(3)
  #print(image.size())
  emb = model(image)
  return emb

In [33]:
def cosine_sim(x1, x2):
  x1 = x1.cpu().detach().numpy()
  x2 = x2.cpu().detach().numpy()
  norm1 = (np.linalg.norm(x1))
  norm2 = (np.linalg.norm(x2))
  return np.dot(x1, x2.T)/(norm1*norm2)

In [34]:
emb1 = get_embeddings('/content/drive/MyDrive/data/t3.jpg')

In [35]:
emb2 = get_embeddings('/content/drive/MyDrive/data/t3.jpg')

In [36]:
cosine_sim(emb1, emb2)

array([[0.99999994]], dtype=float32)

In [42]:
def get_prediction(img_path1, img_path2):
  emb1 = get_embeddings(img_path1)
  emb2 = get_embeddings(img_path2)

  return cosine_sim(emb1, emb2)

In [43]:
get_prediction('/content/drive/MyDrive/data/t3.jpg', '/content/drive/MyDrive/data/t3.jpg')

array([[0.99999994]], dtype=float32)